In [1]:
import pandas as pd
import numpy as np
import importlib
import gc
import sys
import warnings
sys.path.append('./scripts')  
import preprocesamiento
import feature_engineering
import model_autogluon
importlib.reload(preprocesamiento)
importlib.reload(model_autogluon)
importlib.reload(feature_engineering)
warnings.filterwarnings("ignore")

c:\Users\Usuario\.conda\envs\py311lab3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.
Importing plotly failed. Interactive plots will not work.


# Experimento 6: 
- Autogluon: lo más simple posible
- Transformacion boxcox
- Target: TN+2
- Kaggle =  0.268


In [5]:
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor
from scipy.stats import boxcox


# Carga y preparación de datos
df = pd.read_csv("./datasets/periodo_x_producto_con_target.csv", sep=',', encoding='utf-8')
df = df[df['periodo']<201911]
df['target'].fillna(0, inplace=True)  # Rellenar NaN en 'target' con 0




# Agregación y transformación
dfg = df.groupby(['periodo', 'product_id']).agg({'target': 'sum'}).reset_index()
dfg['periodo_dt'] = pd.to_datetime(dfg['periodo'].astype(str), format='%Y%m')
dfg.rename(columns={'product_id':'item_id', 'periodo_dt': 'timestamp'}, inplace=True)
dfg.drop(columns=['periodo'], inplace=True)

# Filtrar productos
productos_ok = pd.read_csv('../../data/raw/product_id_apredecir201912.csv', sep=',')
dfg = dfg[dfg['item_id'].isin(productos_ok['product_id'].unique())]
print(f"Productos únicos a predecir: {len(dfg['item_id'].unique())}")


# Conversión a TimeSeriesDataFrame con verificación
if len(dfg) == 0:
    raise ValueError("El DataFrame está vacío después del filtrado")

try:
    data = TimeSeriesDataFrame.from_data_frame(
        dfg,
        id_column="item_id",
        timestamp_column="timestamp"
    )
    print("TimeSeriesDataFrame creado exitosamente")
    print(f"Número de series temporales: {len(data.item_ids)}")
except Exception as e:
    raise ValueError(f"Error al crear TimeSeriesDataFrame: {str(e)}")



##################
# # Box-Cox transform para estabilizar varianza
data['target'], lambda_param = boxcox(data['target'] + 1)  # +1 para valores positivos
#####################################
  
predictor = TimeSeriesPredictor(
    target='target',
    prediction_length=1,
    freq="M",
    eval_metric="MAPE"
).fit(
    data,
    num_val_windows=5,
    verbosity=2  # Más detalle en logs
)
        

        
preds = predictor.predict(data)
preds['mean'] = (preds['mean'] * lambda_param + 1) ** (1/lambda_param) ### # Inversión de la transformación


        





Productos únicos a predecir: 780
TimeSeriesDataFrame creado exitosamente
Número de series temporales: 780


Frequency 'M' stored as 'ME'
Beginning AutoGluon training...
AutoGluon will save models to 'c:\Users\Usuario\Documents\Universidad\austral\2025\Lab3\Lab3-MCD\notebooks\entregable\AutogluonModels\ag-20250629_203141'
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.11.13
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.26100
CPU Count:          4
GPU Count:          0
Memory Avail:       2.36 GB / 15.89 GB (14.8%)
Disk Space Avail:   399.99 GB / 893.49 GB (44.8%)

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': MAPE,
 'freq': 'ME',
 'hyperparameters': 'default',
 'known_covariates_names': [],
 'num_val_windows': 5,
 'prediction_length': 1,
 'quantile_levels': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'random_seed': 123,
 'refit_every_n_windows': 1,
 'refit_full': False,
 'skip_model_selection': False,
 'target': 'target',
 'verbosity': 2}

train_data with frequency 'IRREG' has been re

In [6]:
preds

,,mean,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9
item_id,timestamp,,,,,,,,,,
20001,2019-11-30,1288.693263,4.561626,4.656189,4.707328,4.744679,4.779654,4.812743,4.847869,4.894652,4.960503
20002,2019-11-30,966.019981,4.409449,4.519611,4.577836,4.617780,4.656963,4.693163,4.733598,4.780318,4.854892
20003,2019-11-30,770.323670,4.304230,4.415450,4.475356,4.517751,4.557530,4.595787,4.637790,4.686601,4.758145
20004,2019-11-30,577.824443,4.162122,4.277475,4.340693,4.385625,4.427214,4.466137,4.510183,4.559120,4.637107
20005,2019-11-30,549.614970,4.129753,4.251633,4.316507,4.358970,4.404060,4.443559,4.487608,4.539100,4.613951
...,...,...,...,...,...,...,...,...,...,...,...
20962,2019-11-30,3.129831,0.856308,0.929388,0.980419,1.026246,1.065302,1.098694,1.144099,1.190615,1.274296
20975,2019-11-30,2.837388,0.770897,0.838544,0.891386,0.938767,0.979413,1.009169,1.054113,1.103943,1.188473
20995,2019-11-30,2.707956,0.719559,0.788882,0.844487,0.896082,0.938172,0.972820,1.022358,1.068038,1.167711


In [8]:
final_df_copy = preds.reset_index()

final_df_copy = final_df_copy[['item_id', 'mean']].rename(columns={'mean': 'tn', 'item_id': 'product_id'})

output_path = "./outputs/predicciones_exp_06v6_autogluon_mean.csv"

final_df_copy.to_csv(output_path, index=False)

In [15]:
df_a = pd.read_csv("./outputs/predicciones_exp_06_autogluon_v1_12kfold.csv", sep=',', encoding='utf-8')
df_b = pd.read_csv("./outputs/prediccion_autogluon_2ventanas.csv", sep=',', encoding='utf-8')
df_a = df_a.merge(df_b, on='product_id', how='inner')
df_a

,product_id,pred_windows_12,tn
0,20001,1458.384452,1302.821147
1,20002,1193.750891,1087.357424
2,20003,771.721654,682.975566
3,20004,561.147217,522.412123
4,20005,602.650778,529.132087
...,...,...,...
775,21263,0.023488,0.030325
776,21265,0.096101,0.054927
777,21266,0.093510,0.057644
778,21267,0.074795,0.058903


In [16]:
df_a['mean'] = (df_a['pred_windows_12'] + df_a['tn']) / 2
df_a = df_a[['product_id', 'mean']].rename(columns={'mean': 'tn'})
output_path = "./outputs/predicciones_exp_06v6_autogluon_mean2.csv"
df_a.to_csv(output_path, sep=',', index=False)

In [17]:
df_a

,product_id,tn
0,20001,1380.602800
1,20002,1140.554158
2,20003,727.348610
3,20004,541.779670
4,20005,565.891432
...,...,...
775,21263,0.026906
776,21265,0.075514
777,21266,0.075577
778,21267,0.066849
